<a href="https://colab.research.google.com/github/LEEHAKJIN-VV/test/blob/main/imageModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kaggle

# from google.colab import files
# files.upload()

In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets list -s dogs-vs-cats

In [ ]:
#!kaggle datasets download -d tongpython/cat-and-dog

In [ ]:
# !unzip cat-and-dog.zip
# !ls

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
print(os.listdir("./data"))
print(len(os.listdir("./data/test1")))

4. 글로벌 변수를 선언 합니다.

앞서 압축 풀어놓은 이미지의 가로크기, 세로크기, 사이즈(128x128)와

채널(칼라 사잔이므로R, G, B세개)을 정의 해 놓습니다.

나중에 convolution연산 시 channel정보가 필요하므로, 미리 정의해 놓습니다.

In [ ]:
FAST_RUN = False
IMAGE_WIDTH = 128
IMAGE_HEIGHT = 128
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS = 3


5. 파일명과 정답 설정

압축 푼 디렉토리 중, train디렉토리에는 dogxxx.jpg, catxxx.jpg등, 개와 고양이를 파일 이름으로 구분짓도록

파일이 저장되어 있습니다.

이를 구분하여 개에 해당되는 파일명과 정답('개', 1)을, 그리고 고양이에 해당되는 파일명과 정답('고양이', 0)을

dataframe에 저장해 놓습니다.

In [ ]:
#Preparig Traning Data
filenames = os.listdir("./data/train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename':filenames,
    'category':categories
})

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df['category'].value_counts().plot.bar()


8. Sample데이터 확인

데이터 디렉토리중 임의로 하나를 뽑아서 확인 합니다.

아베...아니, 개가 잘 표시 되는군요.

In [ ]:
sample = random.choice(filenames)
image = load_img("./data/train/"+sample)
plt.imshow(image)

9. 신경망 모델 구성

모델 구성은...기본적인 룰이 있긴 하지만, 여러번 구조를 바꿔가며,

잘 맞추는 모델이 나올때까지 구조를 변경시키는 것 입니다.

물론, 훌륭하신 석학들이 미리 다 해보고 잘 되는 구조를 공개해 놓은걸 가져다 쓰는게 제일 좋습니다.

 

우리에겐 좋은 구조를 찾기 위한 100대이상의 GPU서버를 가진 연구실이 없기 때문이죠.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense,Activation, BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))   # 2 because we have dokdo or not

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

10. 신경망 구조를 살펴봅니다.

아래와 같은 구조로 생겼는데, 약 천이백만개의 파라미트를 학습해야 하는군요.

학습데이터(개, 고양이사진)이 제일 위로 들어와서 제일 아래로 나가는 구조 입니다.

 

제일 처음 conv2d layer의 shape를 보면 (None, 126, 126, 32) 이렇게 되어 있는데,

가운데 126, 126 숫자 두개가 바로 입력되는 개, 고양이 사진의 가로, 세로 크기 입니다.

 

제일 아래 Dense Layer에 보면 shape에 숫자 2가 있는데, 이게 바로 개냐 고양이냐 하는 두가지 케이스가

있기 때문에 shape가 2로 된 것 입니다.

11. 콜백 정의

음..콜백은 다 아실거고...

우선 조기종료(Early Stopping)와 학습율 하향조정을 위해 두 콜백 클래스를 import합니다.

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

12. Early Stopping 정의

Early Stopping이 뭐냐 하면...

전체 개, 고양이 데이터가지고 조금씩 나눠서 여러차례 학습을 하는데,

잘 학습되다가 어느시점 지나면 정확도가 오히려 떨어지는 case가 발생합니다.

이때 10번까지 참다가, 10번 지나서도 계속 떨어지면 고만 학습하고 그 결과를 내라는 말 입니다.

In [ ]:
# Early Stop
# To prevent over fitting we will stop the learning after 10 epochs and val_loss not decreased
earlystop = EarlyStopping(patience=10)

13. Learning Rate 조정 정의

이 클래스는 학습하는 동안 정확도를 잘 감시하고 있다가 어느 기준이 되면 학습율을 조정해주는 클래스 입니다

In [ ]:
#Learning Rate Reduction
# We will reduce the learning rate when the accuracy not increase for 2 steps

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                           patience=2,
                                           verbos=1,
                                           factor=0.5,
                                           min_lr=0.00001)

14. callback 설정

앞서 정의한 두 콜백 클래스를 callbacks에 담아 놓습니다.

In [ ]:
callbacks = [earlystop, learning_rate_reduction]

15. 개, 고양이를 string으로 변환

아까 파일이름과 개인지 고양이 인지를 저장해놓은 dataframe에서

category를 개인 경우 1로, 고양이 인 경우 0으로 변경해 줍니다.

나중에 one-hot 인코딩으로 변환을 하기 위해서 입니다.

In [ ]:
df['category'] = df['category'].replace({0: 'cat', 1: 'dog'})

16. train-validation 데이터 분리

이제, train데이터중 20%를 쪼개서 학습중 파라미터 검증을 위한 validation셋을 마련해 놓습니다.

In [ ]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

17. train데이터의 분포 확인

train데이터에 존재하는 개와 고양이 수를 그래프로 그려 봅니다.

In [ ]:
train_df['category'].value_counts().plot.bar()

18. validation 분포 확인

아까 잘라낸 20%의 데이터에도 개와 고양이의 분포를 확인 합니다.

In [ ]:
validate_df['category'].value_counts().plot.bar()

19. 학습, 검증데이터의 확인

학습데이터 및 validation의 형상을 확인하고,

한번에 학습할 batch의 size를 설정 합니다.

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size = 15

20. 학습데이터 뻥튀기

아시다 시피 학습은 데이터가 많을수록 잘 될 가능성이 큽니다.

학습데이터를 augmentation해서 수를 늘립니다.

부풀리기는 이미지를 약간 회전시키거나, 줌을 하거나, 상하/좌우 반전을 시키는 방법으로 늘립니다.

In [ ]:
#Train Generator
train_datagen = ImageDataGenerator(
    rotation_range = 15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    "./data/train/",
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

21. Validation데이터 뻥튀기

validation이미지도 마찬가지로 작업을 해줍니다.

In [ ]:
# Validation Generator
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df,
    "./data/train/",
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

22. 샘플 확인

위에 데이터 부풀리기가 잘 되었는지 확인하기 위한 부풀리기를 수행합니다.

In [ ]:
# See how our generator work

example_df = train_df.sample(n=1).reset_index(drop=True)
example_generator = train_datagen.flow_from_dataframe(
    example_df,
    "./data/train",
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    calss_mode='categorical'
)

23. 이미지 확인

작업된 이미지를 확인 해 봅니다.

자세히 보면 같은 개인데, 좌우 반전이나 줌이 되어 있는걸 볼 수 있죠.

In [ ]:
plt.figure(figsize=(12,12))

for i in range(0, 15):
    plt.subplot(5, 3, i+1)
    for X_batch, Y_batch in example_generator:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

24. 학습시작

자, 이제 학습을 해봅시다.

GPU서버에서 한 Epoch당 대략 5분정도가 걸린거 같습니다.

회사에서 지급된 노트북으로는 15분에서 30분 걸립니다. (곱하기 50번을 해야 함)

휴가 갈때 걸어놓고 가시길...

 

학습이 잘 되었고 31번째 Epoch에서 ReduceLROnPlateau콜백이 일어났군요.

1e-50 학습율에 도달한 모양입니다.

In [ ]:
#Fit model
epochs=3 if FAST_RUN else 50
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

25. 모델 저장

학습이 끝나면, 제일먼저 할일은 모델을 저장하는 것입니다.

모델을 저장해 놓으면, 다음에 24번의 오래오래 걸리는 학습 단계를 생략할 수 있습니다.

In [ ]:
####### Save Model
model.save_weights("model.h5")

 

26. 학습 내용 확인

이제 그동안 학습시킨 내용을 확인해봐야겠죠.

train loss, validation loss와 train accuracy, validation accuracy 그래프를 그려봅니다

In [ ]:
# #Virtuallize Training
# fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12,12))
# ax1.plot(history.history['loss'], color='b', label="Training loss")
# ax1.plot(history.history['val_loss'], color='r', label="validation loss")
# ax1.set_xticks(np.arange(1, epochs, 1))
# ax1.set_yticks(np.arange(0, 1, 0.1))

# ax2.plot(history.history['acc'], color='b', label="Training accuracy")
# ax2.plot(history.history['val_acc'], color='r', label="validation accuracy")
# ax2.set_xticks(np.arange(1, epochs, 1))

# legend = plt.legend(loc='best', shadow=True)
# plt.tight_layout()
# plt.show()

In [ ]:
test_filenames = os.listdir("./data/test1/")
test_df = pd.DataFrame({
    'filename' : test_filenames
})

nb_samples = test_df.shape[0]


In [ ]:
test_df.head()
#print(len(test_df))
#print('dog' in test_df.filename)


In [ ]:
test_df.shape

In [ ]:
test_gen = ImageDataGenerator(rescale = 1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "./data/test1/", 
    x_col = 'filename', 
    y_col = None, 
    class_mode = None, 
    target_size = IMAGE_SIZE,
    batch_size = batch_size, 
    shuffle = False
)

In [ ]:
#이 코드는 이미지파일 말고 .ipynb_checkpoints이새끼가 하나 잘못들어가서 지우는 코드
#혹시 이미지말고 다른 파일실수로 들어갈경우 쓰면되고 아니면 안써도됨
# idx = test_df[test_df['filename']=='.ipynb_checkpoints'].index
# test_df = test_df.drop(idx)
# print(test_df.shape)

In [ ]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

In [ ]:
test_df.shape

In [ ]:
print(np.argmax(predict, axis=-1))
predict.shape

In [ ]:
test_df['category'] = np.argmax(predict, axis=-1)

In [ ]:
test_df['category'] = test_df['category'].replace({'dog':1, 'cat':0})

In [ ]:
test_df['category'].values_counts().plot.bar()

In [ ]:
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
  filename = row['filename']
  category = row['category']
  img = load_img("./data/test1/"+filename, target_size=IMAGE_SIZE)
  plt.subplot(6, 3, index+1)
  plt.imshow(img)
  plt.xlabel(filename + '(' + "{}".format(category) + ')')
plt.tight_layout()
plt.show()